In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Input, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
arabic_characters = ['alef', 'beh', 'teh', 'theh', 'jeem', 'hah', 'khah', 'dal', 'thal',
                    'reh', 'zain', 'seen', 'sheen', 'sad', 'dad', 'tah', 'zah', 'ain',
                    'ghain', 'feh', 'qaf', 'kaf', 'lam', 'meem', 'noon', 'heh', 'waw', 'yeh']

In [11]:
x_train = pd.read_csv("ahcd1/csvTrainImages 13440x1024.csv",header=None).to_numpy()
y_train = pd.read_csv("ahcd1/csvTrainLabel 13440x1.csv",header=None).to_numpy()-1 

x_test = pd.read_csv("ahcd1/csvTestImages 3360x1024.csv",header=None).to_numpy()
y_test = pd.read_csv("ahcd1/csvTestLabel 3360x1.csv",header=None).to_numpy()-1

x_train = x_train.reshape(-1,32,32,1)
x_test = x_test.reshape(-1,32,32,1)

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [12]:
def get_model():
    In = Input(shape=(32,32,1))
    x = Conv2D(32, (5,5), padding="same", activation="relu")(In)
    x = Conv2D(32, (5,5), activation="relu")(x)
    x = Conv2D(32, (5,5), activation="relu")(x)
    x = MaxPooling2D((2,2))(x)
    x = BatchNormalization()(x)
    
    x = Conv2D(64, (5,5), padding="same", activation="relu")(x)
    x = Conv2D(64, (5,5), activation="relu")(x)
    x = Conv2D(64, (5,5), activation="relu")(x)
    x = MaxPooling2D((2,2))(x)
    x = BatchNormalization()(x)
    
    x = Flatten()(x)
    x = Dense(128, activation="relu")(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.4)(x)
    
    Out = Dense(28, activation="softmax")(x)
    
    model = Model(In, Out)
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

model = get_model()

In [13]:
datagen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False, 
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False, 
        zca_whitening=False, 
        rotation_range=10, 
        zoom_range = 0.1,
        width_shift_range=0.1, 
        height_shift_range=0.1, 
        horizontal_flip=False,
        vertical_flip=False) 

In [14]:
batch_size = 64
epochs = 50

train_gen = datagen.flow(x_train, y_train, batch_size=batch_size)
test_gen = datagen.flow(x_test, y_test, batch_size=batch_size)

In [15]:
model_checkpoint_callback = ModelCheckpoint(
    filepath="model.hdf5",
    monitor='val_accuracy', 
    verbose=1, 
    save_best_only=True, 
    mode='max')


history = model.fit(train_gen, 
                              epochs = epochs, 
                              steps_per_epoch = x_train.shape[0] // batch_size,
                              validation_data = test_gen,
                              validation_steps = x_test.shape[0] // batch_size,
                              callbacks=[model_checkpoint_callback])

Train for 210 steps, validate for 52 steps
Epoch 1/50
209/210 [============================>.] - ETA: 0s - loss: 2.2186 - accuracy: 0.3123
Epoch 00001: val_accuracy improved from -inf to 0.03576, saving model to model.hdf5
210/210 [==============================] - 75s 357ms/step - loss: 2.2144 - accuracy: 0.3136 - val_loss: 8.0336 - val_accuracy: 0.0358
Epoch 2/50
209/210 [============================>.] - ETA: 0s - loss: 0.9673 - accuracy: 0.6756
Epoch 00002: val_accuracy improved from 0.03576 to 0.13792, saving model to model.hdf5
210/210 [==============================] - 77s 365ms/step - loss: 0.9673 - accuracy: 0.6757 - val_loss: 4.1253 - val_accuracy: 0.1379
Epoch 3/50
209/210 [============================>.] - ETA: 0s - loss: 0.6077 - accuracy: 0.8055
Epoch 00003: val_accuracy improved from 0.13792 to 0.70403, saving model to model.hdf5
210/210 [==============================] - 71s 337ms/step - loss: 0.6076 - accuracy: 0.8059 - val_loss: 0.8890 - val_accuracy: 0.7040
Epoch 4/5

Epoch 29/50
209/210 [============================>.] - ETA: 0s - loss: 0.0966 - accuracy: 0.9732
Epoch 00029: val_accuracy improved from 0.96094 to 0.96304, saving model to model.hdf5
210/210 [==============================] - 68s 322ms/step - loss: 0.0964 - accuracy: 0.9733 - val_loss: 0.1339 - val_accuracy: 0.9630
Epoch 30/50
209/210 [============================>.] - ETA: 0s - loss: 0.0792 - accuracy: 0.9768
Epoch 00030: val_accuracy did not improve from 0.96304
210/210 [==============================] - 69s 328ms/step - loss: 0.0795 - accuracy: 0.9769 - val_loss: 0.1592 - val_accuracy: 0.9588
Epoch 31/50
209/210 [============================>.] - ETA: 0s - loss: 0.0993 - accuracy: 0.9717
Epoch 00031: val_accuracy did not improve from 0.96304
210/210 [==============================] - 70s 335ms/step - loss: 0.1003 - accuracy: 0.9714 - val_loss: 0.2324 - val_accuracy: 0.9372
Epoch 32/50
209/210 [============================>.] - ETA: 0s - loss: 0.0815 - accuracy: 0.9749
Epoch 00032: 

In [18]:
model.save('model.keras')